> study

# a,b,c,d,e 를 표현함에 있어서 3개의 은닉노드면 충분하다. 
- 1개의 은닉노드 -> 2개의 문자를 표현할 수 있음. 
- 2개의 은닉노드 -> 4개의 문자를 표현할 수 있음. 
- 3개의 은닉노드 -> 8개의 문자를 표현할 수 있음.

# torch.nn.RNN(4,2)는 torch.nn.RNNCell(4,2)의 batch 버전이다. 

(for문이 포함된 버전이다)

In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt

/home/csy/anaconda3/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sig = torch.nn.Sigmoid()
soft = torch.nn.Softmax(dim=1)
tanh = torch.nn.Tanh()

In [3]:
def f(txt,mapping):
    return [mapping[key] for key in txt] 

In [4]:
txt = list('ab')*100
txt_x = txt[:-1]
txt_y = txt[1:]

In [5]:
mapping = {'a':[1,0],'b':[0,1]}
x = torch.tensor(f(txt_x,mapping)).float().reshape(-1,2)
y = torch.tensor(f(txt_y,mapping)).float().reshape(-1,2)

In [6]:
class mynet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        ## 우리가 사용할 레이어를 정의 
        self.l1 = torch.nn.Linear(in_features=2,out_features=1,bias=True)
        self.a1 = torch.nn.Tanh()
        self.l2 = torch.nn.Linear(in_features=1,out_features=2,bias=False)
        ## 레이어 정의 끝
    def forward(self,x):
        ## yhat을 어떻게 구할것인지 정의 
        yhat = self.l2(self.a1(self.l1(x)))
        ## 정의 끝
        return yhat

In [7]:
net = mynet()

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [9]:
for epoc in range(5000):
    yhat = net(x)
    loss = loss_fn(yhat,y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

## 임베딩

In [10]:
mapping = {'a':0,'b':1}
x = torch.tensor(f(txt_x,mapping))
y = torch.tensor(f(txt_y,mapping))
x[:5],y[:5]

(tensor([0, 1, 0, 1, 0]), tensor([1, 0, 1, 0, 1]))

In [11]:
class mynet2(torch.nn.Module):
    def __init__(self):
        super().__init__()
        ## 우리가 사용할 레이어를 정의 
        self.l1 = torch.nn.Embedding(num_embeddings=2,embedding_dim=1)
        self.a1 = torch.nn.Tanh()
        self.l2 = torch.nn.Linear(in_features=1,out_features=2)
        ## 레이어 정의 끝
    def forward(self,x):
        ## yhat을 어떻게 구할것인지 정의 
        yhat = self.l2(self.a1(self.l1(x)))
        ## 정의 끝
        return yhat

In [12]:
net = mynet2()

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [14]:
for epoc in range(5000):
    yhat = net(x)
    loss = loss_fn(yhat,y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

## 두 개의 은닉노드 이용

In [15]:
txt = list('abcd')*100
txt_x = txt[:-1]
txt_y = txt[1:]

In [16]:
mapping = {'a':0,'b':1,'c':2,'d':3}
x = torch.tensor(f(txt_x,mapping))
y = torch.tensor(f(txt_y,mapping))
x[:5],y[:5]

(tensor([0, 1, 2, 3, 0]), tensor([1, 2, 3, 0, 1]))

In [17]:
l1=torch.nn.Embedding(num_embeddings=4,embedding_dim=2)

In [18]:
a1 = torch.nn.Tanh()

In [19]:
l2 = torch.nn.Linear(in_features=2,out_features=4)

In [20]:
net = torch.nn.Sequential(
    l1,a1,l2)

In [21]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(net.parameters())

In [22]:
for epoc in range(5000):
    yhat=net(x)
    loss = loss_fn(yhat,y)
    loss.backward()
    optimizr.step()
    optimizr.zero_grad()

In [23]:
hidden = net[:-1](x).data
yhat = soft(net(x)).data

In [24]:
net[:-1]

Sequential(
  (0): Embedding(4, 2)
  (1): Tanh()
)

결과 시각화 코드 10주

## 순환신경망 RNN

10주

In [25]:
txt = list('AbAcAd')*100
txt_x = txt[:-1]
txt_y = txt[1:]
x = torch.tensor(f(txt_x,{'A':0,'b':1,'c':2,'d':3}))
y = torch.tensor(f(txt_y,{'A':0,'b':1,'c':2,'d':3}))
x= torch.nn.functional.one_hot(x).float()
y= torch.nn.functional.one_hot(y).float()

In [26]:
class rNNCell(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.i2h = torch.nn.Linear(4,2) 
        self.h2h = torch.nn.Linear(2,2) 
        self.tanh = torch.nn.Tanh()
    def forward(self,x,hidden):
        hidden = self.tanh(self.i2h(x)+self.h2h(hidden))
        return hidden

In [27]:
torch.manual_seed(43052)
rnncell = rNNCell()

In [28]:
torch.manual_seed(43052)
cook = torch.nn.Linear(2,4) 

In [29]:
loss_fn = torch.nn.CrossEntropyLoss() 
optimizr = torch.optim.Adam(list(rnncell.parameters())+list(cook.parameters()))

In [30]:
T = len(x) 
for epoc in range(100): 
    ## 1~2
    loss = 0 
    ht = torch.zeros(1,2) 
    for t in range(T):
        xt,yt = x[[t]], y[[t]]
        ht = rnncell(xt,ht) 
        ot = cook(ht) 
        loss = loss + loss_fn(ot,yt) 
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

시각화코드는 10주_순환신경망구현1-성공에

## 순환신경망 RNN

In [31]:
txt = list('AbAcAd')*100
txt_x = txt[:-1]
txt_y = txt[1:]
x = torch.nn.functional.one_hot(torch.tensor(f(txt_x,{'A':0,'b':1,'c':2,'d':3}))).float()
y = torch.nn.functional.one_hot(torch.tensor(f(txt_y,{'A':0,'b':1,'c':2,'d':3}))).float()

In [32]:
torch.manual_seed(2) #1 
rnn = torch.nn.RNN(4,3).to("cuda:0") 
cook = torch.nn.Linear(3,4).to("cuda:0")

In [33]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(rnn.parameters())+list(cook.parameters()))

In [36]:
_water = torch.zeros(1,3).to("cuda:0") 
for epoc in range(500):
    ## 1
    hidden,hT = rnn(x.to("cuda:0"),_water) 
    output = cook(hidden) 
    ## 2 
    loss = loss_fn(output,y.to("cuda:0")) 
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

In [37]:
yhat = soft(output)

## LSTM

In [76]:
txt = list('hi?hello!!')*100 
txt_x = txt[:-1]
txt_y = txt[1:]
mapping = {'!':0, '?':1,'h':2,'i':3,'e':4,'l':5,'o':6} 
x= torch.nn.functional.one_hot(torch.tensor(f(txt_x,mapping))).float().to("cuda:0")
y= torch.nn.functional.one_hot(torch.tensor(f(txt_y,mapping))).float().to("cuda:0")

In [77]:
class lSTMCell(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.i2h = torch.nn.Linear(7,16)
        self.h2h = torch.nn.Linear(4,16) 
        self.tanh = torch.nn.Tanh()
    def forward(self,xt,past):
        ht,ct = past 
        ifgo = self.i2h(xt) + self.h2h(ht) 
        it = sig(ifgo[0:4])
        ft = sig(ifgo[4:8])
        gt = tanh(ifgo[8:12])
        ot = sig(ifgo[12:16])
        ct = ft*ct + it*gt
        ht = ot*self.tanh(ct) 
        return ht,ct

In [78]:
lstmcell = lSTMCell().to("cuda:0")
linr = torch.nn.Linear(4,7).to("cuda:0")
loss_fn = torch.nn.CrossEntropyLoss()
optimizr = torch.optim.Adam(list(lstmcell.parameters())+list(linr.parameters()),lr=0.1)

In [79]:
# 초기값셋팅
torch.manual_seed(43052) 
_lstmcell = torch.nn.LSTMCell(7,4).to("cuda:0")
_linr = torch.nn.Linear(4,7).to("cuda:0")
lstmcell.i2h.weight.data = _lstmcell.weight_ih.data 
lstmcell.h2h.weight.data = _lstmcell.weight_hh.data 
lstmcell.i2h.bias.data = _lstmcell.bias_ih.data
lstmcell.h2h.bias.data = _lstmcell.bias_hh.data
linr.weight.data = _linr.weight.data 
linr.bias.data = _linr.bias.data 

In [81]:
for epoc in range(10):
    ## 1
    hidden = []     
    ht = torch.zeros(4).to("cuda:0")
    ct = torch.zeros(4).to("cuda:0")
    for xt,yt in zip(x,y): 
        ht,ct = lstmcell(xt,(ht,ct))
        hidden.append(ht) 
    hidden = torch.stack(hidden)
    output = linr(hidden) 
    ## 2 
    loss = loss_fn(output,y)
    ## 3 
    loss.backward()
    ## 4 
    optimizr.step()
    optimizr.zero_grad()

In [82]:
yhat = soft(output)

In [83]:
yhat

tensor([[3.0012e-06, 1.0435e-04, 9.5849e-04,  ..., 9.9716e-01, 4.8964e-04,
         2.2947e-04],
        [2.5329e-02, 9.4377e-01, 2.7443e-05,  ..., 6.3854e-03, 6.3265e-06,
         1.7445e-02],
        [4.2920e-02, 2.3365e-05, 9.5503e-01,  ..., 9.9885e-04, 9.5255e-04,
         7.4137e-07],
        ...,
        [3.2003e-05, 4.6789e-04, 1.6912e-08,  ..., 3.6898e-04, 6.6607e-04,
         9.9842e-01],
        [9.9740e-01, 1.9907e-03, 4.5425e-05,  ..., 1.1002e-06, 3.5618e-05,
         5.1339e-04],
        [9.6844e-01, 1.0125e-03, 2.8011e-02,  ..., 9.3604e-05, 1.9250e-03,
         7.5295e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# 12주 할 차례